In [ ]:
import numpy as np
import pandas as pd
import glob
from skimage.io import imread
import random
import seaborn as sns
from skimage.transform import resize
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,Flatten,Dropout,Dense, BatchNormalization, Activation, Conv2DTranspose, Concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l1_l2,l1,l2
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import f_classif

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/DeepLearning/

In [ ]:
ls

In [ ]:
infile=open('partially_covered.pkl', 'rb')
X_partially_covered=pickle.load(infile)
infile.close()

infile=open('not_face.pkl', 'rb')
X_not_face=pickle.load(infile)
infile.close()

infile=open('not_covered.pkl', 'rb')
X_not_covered=pickle.load(infile)
infile.close()

infile=open('fully_covered.pkl', 'rb')
X_fully_covered=pickle.load(infile)
infile.close()

In [ ]:
X_partially_covered=np.array(X_partially_covered)
X_not_face=np.array(X_not_face)
X_not_covered=np.array(X_not_covered)
X_fully_covered=np.array(X_fully_covered)

print(f"Partially covered shape: {X_partially_covered.shape}")
print(f"Not face shape: {X_not_face.shape}")
print(f"Not covered shape: {X_not_covered.shape}")
print(f"Fully covered shape: {X_fully_covered.shape}")

In [ ]:
X=np.concatenate((X_not_covered,X_fully_covered),axis=0)
Y=np.concatenate((np.repeat(0, X_not_covered.shape[0]),np.repeat(1, X_fully_covered.shape[0])),axis=0).reshape(-1,1)
one = OneHotEncoder()
Y=one.fit_transform(Y).toarray()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1,random_state=0)
X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size=0.15/(1-0.1),random_state=1)

X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

In [ ]:
checkpoints = ModelCheckpoint(
                            "checkpoint_mejor_modelo_v2.h5",
                            monitor='val_loss', ## que funcion monitorea para crear el checkpoint
                            verbose=10, ## imprima que guardo un checkpoint
                            save_best_only=True, ## solo guarde el mejor model
                            save_weights_only=False, ## Nos guarda la arquitectura y los pesos
                            mode="min", ### Cuando guardar el checkpoint, en este caso, cada nuevo minimo en la funcion de val_loss
                            save_freq='epoch') ### Cada cuando guarda el checkpoint


In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(64,64,1))) ## las dimensiones de la imagen
model.add(Conv2D(filters= 16 , kernel_size=(2,2),strides=(1,1), padding="same", activation = "relu" ))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid')) 
model.add(Dropout(0.2))
model.add(Conv2D(filters= 32 , kernel_size=(3,3),strides=(1,1), padding="same", activation = "sigmoid" ))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid'))  
model.add(Conv2D(filters= 64 , kernel_size=(2,2),strides=(1,1), padding="same", activation = "relu" ))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid')) 
model.add(Flatten()) ## convetimos un tensor de cualesquiera dimensiones a un vector 
model.add(Dropout(0.2))
model.add(Dense(units=256, activation ="relu"))
model.add(Dense(units=128, activation ="sigmoid"))
model.add(Dense(units=64, activation ="relu", kernel_regularizer= l1_l2(0.01,0.01)))
model.add(Dense(2, activation="softmax"))

model.compile(loss=BinaryCrossentropy(), optimizer=Adam(0.0001,), metrics= ["accuracy"])

model.summary()

In [ ]:
history = model.fit(X_train,Y_train, epochs= 100, batch_size =64, validation_data=(X_val,Y_val),callbacks=[checkpoints])


In [ ]:
# Plot loss
plt.figure(figsize=(20, 4))

plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(model.history.history['loss'], label='Training', linewidth=2)
plt.plot(model.history.history['val_loss'], label='Validation', linewidth=2)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(model.history.history['accuracy'], label='Training', linewidth=2)
plt.plot(model.history.history['val_accuracy'], label='Validation', linewidth=2)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.show()

In [ ]:
best_model = load_model("checkpoint_mejor_modelo_v2.h5")

In [ ]:
test_loss = best_model.evaluate(x=X_test, y=Y_test)

In [ ]:
Y_test_hat = best_model.predict(x=X_test)

Y_test_hat_cat = np.argmax(Y_test_hat, axis=1)
Y_test_cat = np.argmax(Y_test, axis=1)
cm=confusion_matrix(y_true=Y_test_cat, y_pred=Y_test_hat_cat)

print(cm)
print("\n")
print(np.round(np.transpose(np.transpose(cm)/cm.sum(axis=1)),3)*100)

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True)

In [ ]:
sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
ypred = model.predict(X_test)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(Y_test[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

In [ ]:
label=['Not covered', 'Fully covered']
imidx = random.sample(wrongindex, k=9)# replace with 'wrongindex'

nrows = 3
ncols = 3
fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(X_test[imidx[n]], cmap='gray')
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(Y_test[imidx[n]])]))
            n += 1

plt.show()

In [ ]:
label=['Not covered', 'Fully covered']
imidx = random.sample(accurateindex, k=9)# replace with 'wrongindex'

nrows = 3
ncols = 3
fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(X_test[imidx[n]], cmap='gray')
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(Y_test[imidx[n]])]))
            n += 1

plt.show()

In [ ]:
Ypred = model.predict(X_test)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(Y_test, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(Ytrue, Ypred))